In [25]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import ensemble, metrics, model_selection, naive_bayes

import lightgbm as lgb
from tqdm import tqdm
from ast import literal_eval

In [26]:
import numpy as np
# import seaborn as sns
from multiprocessing import Pool

num_partitions = 10 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [2]:
X_train = pd.read_csv('df_train_stat_2.csv')
X_test = pd.read_csv('df_test_stat_2.csv')

In [4]:
X_train

Unnamed: 0      ID  Код_диагноза  \
0                0       0          1327   
1                1       1          2469   
2                2       2          1701   
3                3       3          2408   
4                4       4          3324   
5                5       5          3330   
6                6       6          1987   
7                7       7           446   
8                8       8          2476   
9                9       9           996   
10              10      10           971   
11              11      11          1503   
12              12      12          1502   
13              13      13          2044   
14              14      14           679   
15              15      15          2099   
16              16      16          2044   
17              17      17           790   
18              18      18          2044   
19              19      19          2469   
20              20      20          2468   
21              21      21           314   
22              22      22           312   
23              23      23          2088   
24              24      24          2898   
25              25      25          2448   
26              26      26          2389   
27              27      27          3324   
28              28      28          3334   
29              29      29          2454   
...            ...     ...           ...   
283056      283056  283056           442   
283057      283057  283057           284   
283058      283058  283058          1401   
283059      283059  283059          2086   
283060      283060  283060          1329   
283061      283061  283061          2476   
283062      283062  283062           105   
283063      283063  283063          3334   
283064      283064  283064           108   
283065      283065  283065          2517   
283066      283066  283066           502   
283067      283067  283067            35   
283068      283068  283068          1329   
283069      283069  283069          2044   
283070      283070  283070          2210   
283071      283071  283071          1685   
283072      283072  283072          2108   
283073      283073  283073          2044   
283074      283074  283074          2042   
283075      283075  283075           455   
283076      283076  283076          3141   
283077      283077  283077          3336   
283078      283078  283078          3227   
283079      283079  283079          2109   
283080      283080  283080          1350   
283081      283081  283081           441   
283082      283082  283082          3334   
283083      283083  283083           867   
283084      283084  283084          1304   
283085      283085  283085          2296   

                                                  Диагноз  Возраст  Пол  \
0                           ['острый', 'ларингофарингит']       29    2   
1                 ['подострый', 'хронический', 'вагинит']       45    2   
2       ['аллергический', 'контактный', 'дерматит,', '...       63    2   
3                                       ['баланопостить']       43    2   
4                      ['общий', 'медицинский', 'осмотр']       29    1   
5       ['другой', 'специальный', 'осмотр', 'обследова...       50    2   
6       ['поражение', 'мениск', 'результат', 'старое',...       42    2   
7                   ['тиреотоксикоз', 'диффузный', 'зоб']       85    1   
8       ['вагинит,', 'вульвит', 'вульвовагинит', 'инфе...       28    2   
9                                    ['наружный', 'отит']       38    1   
10                                        ['астигматизм']       25    2   
11                                          ['диспепсия']       51    1   
12                     ['гастродуоденит', 'неуточненный']       52    2   
13            ['остеохондроз', 'позвоночник', 'взрослый']       56    2   
14                            ['мигрень', 'неуточненный']       27    2   
15                   ['другой', 'уточнить', 'дорсопатия']       31    2   
16  

In [29]:
X_train.drop(['Unnamed: 0'], axis=1, inplace=True)

In [31]:
X_test.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [32]:
X_train = X_train.set_index('ID')
X_test = X_test.set_index('ID')

In [22]:
X_train.apply()

['аллергический', 'контактный', 'дерматит,', 'причина', 'уточнить']

In [5]:
X_train['all'] = (X_train['Анамнез заболевания'].values.tolist() + 
                  X_train['Диагноз'].values.tolist() +
                  X_train['Общее состояние'].values.tolist() +
                  X_train['аллергия'].values.tolist() + 
                  X_train['Внешний осмотр'].values.tolist())

X_test['all'] = (X_test['Анамнез заболевания'].values.tolist() + 
                  X_test['Диагноз'].values.tolist() +
                  X_test['Общее состояние'].values.tolist() +
                  X_test['аллергия'].values.tolist() + 
                  X_test['Внешний осмотр'].values.tolist())

ValueError: Length of values does not match length of index

In [ ]:
def tfidfAll(X_train, X_test):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['all'].values.tolist() + X_test['all'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['all'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['all'].values.tolist())
    return train_tfidf, test_tfidf, full_tfidf


    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_tfidf_MNB(X_train, X_test, Y_train, func):
    train_tfidf, test_tfidf, full_tfidf = func(X_train, X_test)
    
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 2])
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)  
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y[:, 1].reshape((-1, 1))
        cv_scores.append(metrics.log_loss(val_y.values.reshape((-1, 1)), pred_val_y))
    
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train, pred_full_test

In [ ]:
%%time
for f in [tfidfAll]:
    pred_train, pred_test = do_tfidf_MNB(X_train, X_test, Y_train, f)
    X_train["tfidf_nb_eap_{}".format(f.__name__)] = pred_train[:,0]
    X_test["tfidf_nb_eap_{}".format(f.__name__)] = pred_test[:,0]